## Meduza.io parsing

In [4]:
import requests
import json
import time
import os

In [5]:
%run tools.py

In [6]:
stream = 'https://meduza.io/api/v3/search?chrono=news&page={page}&per_page=30&locale=ru'
social = 'https://meduza.io/api/v3/social'
USER_AGENT = 'Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/47.0.2526.111 YaBrowser/16.2.0.3539 Safari/537.36'
headers = {'User-Agent' : USER_AGENT }

In [7]:
def get_page_data(page):
    ans = requests.get(stream.format(page = page), headers=headers).json()
    ans_social = requests.get(social, params = {'links' : json.dumps(ans['collection'])}, headers=headers).json()
    documents = ans['documents']
    for url, data in documents.iteritems():
        try:
            data['social'] = ans_social[url]['stats']
        except KeyError:
            continue
    with open('res_dump/page{pagenum:03d}_{timestamp}.json'.format(
        pagenum = page, timestamp = int(time.time())
    ), 'wb') as f:
        json.dump(documents, f, indent=2)

In [8]:
%job [get_page_data(page) for page in log_progress(range(200))]
%job [get_page_data(page) for page in log_progress(range(200,400))]
%job [get_page_data(page) for page in log_progress(range(400,600))]
%job [get_page_data(page) for page in log_progress(range(600,752))]

Starting job # 0 in a separate thread.
Starting job # 2 in a separate thread.
Starting job # 3 in a separate thread.
Starting job # 4 in a separate thread.


In [10]:
jobs.status()

Running jobs:
0 : [get_page_data(page) for page in log_progress(range(200))]
2 : [get_page_data(page) for page in log_progress(range(200,400))]
3 : [get_page_data(page) for page in log_progress(range(400,600))]
4 : [get_page_data(page) for page in log_progress(range(600,752))]



In [ ]:
results_combined = {}
for f in os.listdir("res_dump"):
    if f.endswith(".json"):
        results_combined.update(json.load(open(os.path.join('res_dump', f))))



In [ ]:
json.dumps(results_combined, open('database.json', 'w'))